<a href="https://colab.research.google.com/github/HWMV/News_summary/blob/main/News_summary_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer learning
# Transformers Bert pre-trained model

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.6 MB/s eta 0:00:00


In [7]:
from importlib.metadata import version
import nltk
import tensorflow
import pandas

print(nltk.__version__)
print(tensorflow.__version__)
print(pandas.__version__)

3.8.1
2.14.0
1.5.3


In [8]:
import urllib.request
import pandas as pd

# 데이터셋을 다운로드합니다.
urllib.request.urlretrieve("https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv", filename="news_summary_more.csv")

# 데이터셋을 로드합니다.
# 추출적 요약에 사용할 데이터
data_test = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')
# 전처리 하여 추상적 요약에 사용할 데이터
data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

In [9]:
# data sample Check
data.sample(10)

,headlines,text
6956,Salman Khan spoke to me about it: Sunil on wor...,Sunil Grover has said that Salman Khan spoke t...
18853,Maharashtra woman held for posing as cop to ex...,A 33-year-old woman was arrested for allegedly...
50815,"Jaya hosts dinner for female nominees, winners...",Veteran actress Jaya Bachchan hosted a dinner ...
18806,"Tanushree Dutta, sister Ishita to be in Bigg B...",Tanushree Dutta and her sister Ishita Dutta ha...
43056,DMK chief Stalin asks TN govt to stop VHP's Ra...,DMK Acting President MK Stalin has demanded th...
18684,Writer's imagination must enjoy freedom: SC sa...,The Supreme Court on Wednesday dismissed a pet...
46538,Student expelled after parents ask for time to...,A private school in Kolkata allegedly rusticat...
47830,Chahal shares 'bromance' pic with Kuldeep on V...,Following India's first-ever series win in Sou...
50400,"Govt saved Ã¢ÂÂ¹65,000 crore through DBT sche...",NITI Aayog CEO Amitabh Kant has said the Direc...
38121,Were made to undergo pregnancy test: MP police...,"Female police aspirants in Madhya Pradesh, who..."


In [16]:
# from transformers import BartTokenizer, BartForConditionalGeneration

# # 모델과 토크나이저 초기화
# model_name = 'facebook/bart-large-cnn'  # 요약에 적합한 BART 모델
# tokenizer = BartTokenizer.from_pretrained(model_name)
# model = BartForConditionalGeneration.from_pretrained(model_name)

# def generate_summary(text):
#     # 텍스트를 토큰으로 인코딩하고 요약 생성
#     inputs = tokenizer([text], max_length=200, return_tensors='pt', truncation=True)
#     summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150, early_stopping=True)
#     return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 데이터셋의 각 텍스트에 대해 요약 생성
# data['generated_summary'] = data_test['text'].apply(generate_summary)

In [22]:
# 연산이 오래 걸려서 샘플링으로 축소
from transformers import BartTokenizer, BartForConditionalGeneration

# 모델과 토크나이저 초기화
model_name = 'facebook/bart-large-cnn'  # 요약에 적합한 BART 모델
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# min_length=30, length_penalty=2.0 최소설정, 페널티 부과 (max_length에 가깝게 도움)
def generate_summary(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=150, truncation=True, padding="max_length")
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=30, min_length=30, length_penalty=2.0, early_stopping=True)
    return [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
# 데이터셋의 각 텍스트에 대해 요약 생성


In [17]:
# 데이터셋에서 일부분만 가져와서 요약 생성 (예: 처음 10개)
sample_data = data_test['text'][:10].tolist()
summaries = generate_summary(sample_data)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1298: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


In [18]:
# 생성된 요약문을 데이터 프레임에 저장하고 출력
for i, summary in enumerate(summaries):
    print(f"원본 내용 {i+1}: {sample_data[i]}")
    print(f"생성된 요약 {i+1}: {summary}\n")

원본 내용 1: Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program and upGrad's 360-degree career support helped him transition to a Data Scientist at Tech Mahindra with 90% salary hike. upGrad's Online Power Learning has powered 3 lakh+ careers.
생성된 요약 1: Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys. The program and upGrad's 360-degree career

원본 내용 2: Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coins. Users get one CRED coin per rupee of bill paid, which can be used to avail rewards from brands like Ixigo, BookMyShow, UberEats, Cult.Fit and more.
생성된 요약 2: Kunal Shah's credit card bill payment platform, C

In [23]:
# 성능 테스트 겸 LSTM, Attention에서 사용한 텍스트 샘플
long_text = """
New Zealand defeated India by 8 wickets in the fourth ODI at Hamilton on Thursday to win their first match of the five-match ODI series. India lost an international match under Rohit Sharma's captaincy after 12 consecutive victories dating back to March 2018. The match witnessed India getting all out for 92, their seventh lowest total in ODI cricket history.
"""
print(generate_summary(long_text))
# ratio=0.4 하이퍼 파라미터는 나중에 함수에 추가해서 해보자. 런타임 오래걸려..ㅠ


["New Zealand defeated India by 8 wickets in the fourth ODI at Hamilton. India lost an international match under Rohit Sharma's captain"]


# 성능 비교 (Bert vs Summa summary)
본문 번역 : 뉴질랜드는 목요일 해밀턴에서 열린 네 번째 ODI에서 인도를 8위켓으로 꺾고 5경기 ODI 시리즈의 첫 경기에서 승리했습니다. 인도는 2018년 3월부터 12연승을 거둔 후 Rohit Sharma의 주장 아래 국제 경기에서 패했습니다. 이 경기에서 인도는 ODI 크리켓 역사상 7번째로 낮은 합계인 92점을 기록했습니다.

---
# summma
1. 요약\
Later Try..

---
# Bert (pre-trained model)

1. 요약 \
번역 : ["뉴질랜드는 해밀턴에서 열린 네 번째 ODI에서 인도를 8위켓으로 이겼습니다. 인도는 2018년 3월부터 12연승을 거둔 후 Rohit Sharma의 주장 아래 국제 경기에서 패했습니다. 이 경기에서 인도는 92점을 기록했습니다."]

2. 요약 (max_length 30, min, penalty)\
번역 : ["뉴질랜드는 해밀턴에서 열린 네 번째 ODI에서 인도를 8위켓으로 이겼습니다. 인도는 Rohit Sharma의 주장이 이끄는 국제 경기에서 패했습니다."]